In [3]:
from os import urandom
import requests
from aes import aes_cbc_encrypt, aes_cbc_decrypt
from padding import pkcs7_pad, pkcs7_unpad
from utils import xor

message || IV || MAC  
message = from=#{from_id}&to=#{to_id}&amount=#{amount}

In [4]:
from_id = 1
to_id = 2
amount = 1000000
# transactions is a json list of transactions in the form of:
# to:amount(;to:amount)*
transactions = f"{to_id}:{amount};5:1000"
MSG = f'from=#{from_id}&tx_list=#{transactions}'.encode()
MSG_padded = pkcs7_pad(MSG, 16)
print(MSG_padded)
KEY = b'YELLOW SUBMARINE'
IV = b'\x00'*16

MAC = aes_cbc_encrypt(MSG_padded, KEY, IV)[-16:]
r = requests.get('http://localhost:8000/transfer', params={'hexm': (MSG + MAC).hex()})
print(MSG + MAC)
print(r.json())

b'from=#1&tx_list=#2:1000000;5:1000\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'
b'from=#1&tx_list=#2:1000000;5:1000\xa0T{\xb4\x86\xbb\xaad\x03\x12\xbb\xa6\xf2\xbf\xd0\xdb'
{'status': 'success', 'from': '1', 'transactions': [{'to': '2', 'amount': '1000000'}, {'to': '5', 'amount': '1000'}]}


In [5]:
from utils import get_blocks
captured_MSG = b'from=#1&tx_list=#2:1000000;5:1000\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\xa0T{\xb4\x86\xbb\xaad\x03\x12\xbb\xa6\xf2\xbf\xd0\xdb'
MAC = captured_MSG[-16:]
MSG = captured_MSG[:-16]
print(MSG)
# we want to add a transaction to the list of transactions to account 3, with amount 1000000
# so we need to do a hash extension attack on the MAC
# wanted_MSG = b'from=#1&tx_list=#2:1000000;5:1000;3:1000000'
m2 = b';3:1000000'
m_new = MSG + m2
m_new_padded = pkcs7_pad(m_new, 16)
print(m_new_padded)

b'from=#1&tx_list=#2:1000000;5:1000\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'
b'from=#1&tx_list=#2:1000000;5:1000\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f;3:1000000\x06\x06\x06\x06\x06\x06'


Cn ^ m2[:16]  
Assuming the server gives us a way to hash our input if our account number is verified. We can use this.  
new mac(m1 + m2) = mac(mac(m1) ^ m2[-16:])

In [6]:
# step 1: xor the MAC with the last 16 bytes of the new message
MAC_test = xor(MAC, m_new_padded[-16:])
# step 2: get the mac of the xor'd message
MAC_test = aes_cbc_encrypt(MAC_test, KEY, IV)
r = requests.get('http://localhost:8000/transfer', params={'hexm': (m_new + MAC_test).hex()})
print(m_new + MAC_test)
print(r.json())

b'from=#1&tx_list=#2:1000000;5:1000\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f;3:1000000pyZ\x81\x1bS\x06\xf0#"\xd2t\x85p\xe6\x00'
{'status': 'success', 'from': '1', 'transactions': [{'to': '2', 'amount': '1000000'}, {'to': '5', 'amount': '1000\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'}, {'to': '3', 'amount': '1000000'}]}
